In this assignment, you'll be working with functions, generators, and decorators.

## Instructions

- Inside the solutions folder (`programming3-class/assignment_functions/solutions/`), create a folder with your name lowercase (for e.g., if your name is Anier, then your folder would be named `anier`).
- Copy the given notebook inside the corresponding folder (in my case, I would have to copy it to `programming3-class/assignment_functions/solutions/anier/`) and solve the tasks.
- Each of the tasks contains a line saying `raise NotImplementedError`. This is there in purpose. Once you've implemented a task, remove that line on that task.
- Once you're done, make a PR with your commits.

## Task 1

Write a function with documentation in a docstring (no need for a complicated documentation, just something very simple describing the function).
- The function should contain all types of arguments (positional, positional with defaults, arbitrary args, keyword args, arbitrary keyword args).

In [3]:
def my_function(pos1, pos2, pos_with_default='default', *args, kw1, kw2='kw_default', **kwargs):
    """    
    :pos1: A mandatory positional argument.
    :pos2: Another mandatory positional argument.
    :pos_with_default: A positional argument with a default value.
    :args: A list of variable-length positional arguments.
    :kw1: A mandatory keyword argument.
    :kw2: A keyword argument with a default value.
    :kwargs: A dictionary of variable-length keyword arguments.
    """
    return pos1, pos2, pos_with_default, args, kw1, kw2, kwargs

print(my_function(1, 2, kw1='kw1'))

(1, 2, 'default', (), 'kw1', 'kw_default', {})


## Task 2

A number is prime if it has 2 divisors exactly.

The first few primes are: 2, 3, 5, 7, 11, ...



Make a function that checks if a number is prime or not.

In [2]:
def is_prime(n: int) -> bool:
    """
    Checks if n is prime or not
    """
    if n <= 1:
        return False
    elif n <= 3:
        return True
    elif n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True
is_prime(5)

True


## Task 3

Implement the following function according to the docstring provided.

Read https://docs.python.org/3.11/library/inspect.html in order to see how to get each requested information.

In [5]:
import inspect

def inspect_function(func):
    """
    Takes another function as an argument (but not built-in) 
    and print the following data: 
    the name of the analyzed function, 
    the name of all the arguments it takes 
    and their types (positional, keyword, etc.)
    """
    print("Function name:", func.__name__)

    signature = inspect.signature(func)

    for parameter in signature.parameters.values():
        print(parameter.name)
        print(parameter.kind)
        if parameter.default != inspect.Parameter.empty:
            print(parameter.default)
        if parameter.annotation != inspect.Parameter.empty:
            print(parameter.annotation)
        print()  

def example_function(a, b=10, *, c: int):
    pass

inspect_function(example_function)

Function name: example_function
a
POSITIONAL_OR_KEYWORD

b
POSITIONAL_OR_KEYWORD
10

c
KEYWORD_ONLY
<class 'int'>



## Task 4

The following function isn't working properly. 
Fix it so that it prints the current datetime with a message.

In [6]:
from datetime import datetime
from time import sleep
 
def my_time_now(msg):
    dt = datetime.now()  
    print(msg, dt)

In [7]:
# simple tests :)
my_time_now('The time is now: ')
sleep(1)
my_time_now('The time is now: ')
sleep(1)
my_time_now('The time is now: ')

The time is now:  2024-02-12 00:23:52.699108
The time is now:  2024-02-12 00:23:53.702584
The time is now:  2024-02-12 00:23:54.705927


## Task 5

Make a function that returns at most `max_count` values of a given generator.

In [11]:
def limit(input_generator, max_count):
    """
    Generator that returns not more than max_count values of the input_generator.
    """
    count = 0
    for item in input_generator:
        if count < max_count:
            yield item
            count += 1
        else:
            break

## Task 6

Write a generator for an infinite sequence of numbers from the Pascal's triangle. The sequence look like this: `1 1 1 1 2 1 1 3 3 1 1 4 6 4 1 1 5 10 10 5 1 1 6 15 20 15 6 1 1 7 21 35 35 21 7 1 1 8 28 56 70 56 28 8 1 1 9 36 84 126 126 84 36 9 1 ... '

Test it with a generator from the previous task

In [12]:
def pascals_triangle():
    row = [1]
    while True:
        for num in row:
            yield num
        # Calculate the next row
        row = [x + y for x, y in zip([0] + row, row + [0])]
        
pascal_gen = pascals_triangle()
limited_pascal = limit(pascal_gen, 10) 
for number in limited_pascal:
    print(number, end=' ')



1 1 1 1 2 1 1 3 3 1 

## Task 7

Write a `merge_sorter` generator that merges sorted sequences of integers.

The generator takes an arbitrary number of arguments. The argument can be any iterable, including another generator. It is guaranteed that each argument is a sequence of integers, sorted in non-decreasing order.

In [8]:
def merge_sorter(*args):
    iterators = [iter(seq) for seq in args]
    
    current_values = [next(iterator, None) for iterator in iterators]
    
    while any(value is not None for value in current_values):
        min_value = min(value for value in current_values if value is not None)
        
        yield min_value
        
        index = current_values.index(min_value)
        current_values[index] = next(iterators[index], None)

sequence1 = [1, 3, 5, 7]
sequence2 = [2, 4, 6, 8]
sequence3 = [0, 9, 10]
merged_sequence = merge_sorter(sequence1, sequence2, sequence3)

print(list(merged_sequence))  


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Task 8

Write the decorator proﬁler, which, when calling a function, will store in its attributes (not to be confused with arguments) the time of its execution (in seconds, it can be fractional) and the number of recursive calls that occurred during execution. Name the attributes `last_time_taken` and `calls`. It is forbidden to use global variables. The decorator must behave in a decent manner, that is, it must not overwrite the function's documentation.

For tests write [Ackemann function](https://en.wikipedia.org/wiki/Ackermann_function).

In [10]:
import time

def profiler(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        wrapper.calls = 0
        
        def profiled_func(*args, **kwargs):
            wrapper.calls += 1
            return func(*args, **kwargs)
        
        result = profiled_func(*args, **kwargs)
        end_time = time.time()
        wrapper.last_time_taken = end_time - start_time
        return result
    
    wrapper.__doc__ = func.__doc__
    
    return wrapper

@profiler
def ackermann(n: int, m: int) -> int:
    if n == 0:
        return m + 1
    elif m == 0:
        return ackermann(n - 1, 1)
    else:
        return ackermann(n - 1, ackermann(n, m - 1))

print(ackermann(3, 4))
print("time:", ackermann.last_time_taken)
print("calls:", ackermann.calls)


125
time: 0.01887822151184082
calls: 1
